## SAS로 진행됐던 정규화 과정을 파이썬으로 옮겨본 방식

In [1]:
import numpy as np
import pandas as pd
np.random.seed(1234)

#생성된 난수에 대해 정규화를 진행한다.
data1 = pd.DataFrame(
    data = {'grade' : np.random.rand(10000)*100})
data1.index.name = 'index'
data1['group'] = 0
data1.to_csv('C:/Users/jiho0/OneDrive/바탕 화면/SAS/SAS_CODE/참고자료/sample_data/sample.txt', sep = ' ', encoding = 'cp949')

In [2]:
from scipy.stats import norm

for i in np.arange(10,16,0.00001) : 
    if norm(50,i).cdf(x=0) >= 0.0005 :
        print(i)
        break

15.179949999803899


In [3]:
from scipy.stats import norm
pctl = []
# i = 15.179949999803899
for k in range(0,102,2) : 
    pctl.append(round(norm(50,i).cdf(x = k), ndigits = 4))

print(pctl)

[0.0005, 0.0008, 0.0012, 0.0019, 0.0028, 0.0042, 0.0062, 0.0089, 0.0126, 0.0175, 0.0241, 0.0326, 0.0434, 0.0569, 0.0736, 0.0938, 0.1179, 0.1459, 0.1782, 0.2146, 0.255, 0.2991, 0.3463, 0.3961, 0.4476, 0.5, 0.5524, 0.6039, 0.6537, 0.7009, 0.745, 0.7854, 0.8218, 0.8541, 0.8821, 0.9062, 0.9264, 0.9431, 0.9566, 0.9674, 0.9759, 0.9825, 0.9874, 0.9911, 0.9938, 0.9958, 0.9972, 0.9981, 0.9988, 0.9992, 0.9995]


In [4]:
cut1 = data1.quantile(q = pctl)

cut1['group'] = np.arange(1,52,1)
# cut1 = cut1.sort_values(by = ['grade'], axis = 0, ascending = False)

cut1.head(5)

,grade,group
0.0005,0.046145,1
0.0008,0.068330,2
0.0012,0.131785,3
0.0019,0.224374,4
0.0028,0.407392,5


In [5]:

cut1 = data1.quantile(q = pctl)
cut1['group'] = np.arange(1,52,1)

# cut1 = cut1.sort_values(by = ['grade'], axis = 0, ascending = False)
print(cut1.head(5))

           grade  group
0.0005  0.046145      1
0.0008  0.068330      2
0.0012  0.131785      3
0.0019  0.224374      4
0.0028  0.407392      5


In [6]:
cut2 = cut1.shift(-1)
cut2['group'] = cut2['group'].fillna(52)
cut2['grade'] = cut2['grade'].fillna(100) 
cut2.columns = ['upper', 'group']

cut3 = cut1.join(cut2['upper'])

In [7]:
for a in data1.index : 
    val = data1.loc[a, 'grade']
    for b in cut3.index : 
        cut_upper = cut3.loc[b, 'upper']
        cut_lower = cut3.loc[b, 'grade']
        group = cut3.loc[b, 'group']
        if val > cut_lower and val <= cut_upper : 
            data1.loc[a, 'group'] = group

In [8]:
data2 = data1.groupby('group').max()
data2.columns = ['max'] 


In [9]:
max_lag = data2['max'].shift(1)
max_lag.fillna(0)
data3 = pd.concat([data2, max_lag], axis = 1)
data3.columns = ['max', 'max_lag']
del max_lag

In [10]:
base1 = np.arange(0,100,2)
base1 = base1.tolist()
base1.insert(0, np.nan)
base1.append(np.nan)
base2 = pd.Series(base1)

data4 = pd.concat([data3, base2], axis = 1)
data4.sort_index(inplace = True)
data4.columns = ['max', 'max_lag', 'base']
del base1, base2

In [11]:
data4['gradient'] = 2 / (data4['max'] - data4['max_lag'])

In [12]:
for a in data1.index : 
    val  = data1.loc[a, 'grade']
    for b in data4.index : 
        max_ = data4.loc[b,'max']
        max_lag = data4.loc[b,'max_lag']
        base = data4.loc[b,'base']
        gradient = data4.loc[b,'gradient']
            
        if val >max_lag and val <= max_ :
            if b not in [0, 51] : 
                data1.loc[a, 'grade_regul'] = base + gradient * (val - max_lag)
            elif b in [0] : 
                data1.loc[a, 'grade_regul'] = 0
            else : 
                data1.loc[a, 'grade_regul'] = 100

In [13]:
data1['grade_regul'] = data1['grade_regul'].fillna(0)
data_fin = data1.sort_values(by = ['grade_regul'], axis = 0)
